In [1]:
import torch
import pandas as pd
from transformers import pipeline, Conversation


c:\Users\1027d\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chat_model = pipeline(task = "conversational", model = "facebook/blenderbot-400M-distill")

In [13]:
user_inputs = []
template = [{"role": "system", "content" : "You must ask about something related on user's personality."},
            ]


while 1:
    user_input = input()
    if user_input.lower() == "done":
        break
    print("You: ", user_input, flush=True)
    user_inputs.append(user_input)
    template.append({"role":"user", "content": f"{user_input}"})
    if len(template) > 5:
        template.pop(1)
        template.pop(1)
        template.pop(1)
    try:
        bot_response = chat_model(template, max_length = 128)[-1]["content"]
        print("Bot: ", bot_response)
        template.append({"role": "assistant", "content": f"{bot_response}"})
    except:
        template.pop(-1)
        print("You typed too many words, please make your sentence shorter.")
